In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [3]:
movie = pd.read_csv('/kaggle/input/movielens-25m-dataset/ml-25m/movies.csv')
ratings = pd.read_csv('/kaggle/input/movielens-25m-dataset/ml-25m/ratings.csv')

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
import pandas as pd

ratings_df = pd.read_csv('ml-25m/ratings.csv')
movies_df = pd.read_csv('ml-25m/movies.csv')

movies_ratings_df = pd.merge(ratings_df, movies_df, on="movieId")


In [6]:
combine_movie_rating = movies_ratings_df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()

,title,totalRatingCount
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",179
2,#1 Cheerleader Camp (2010),9
3,#Captured (2017),2
4,#Female Pleasure (2018),3


In [7]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,timestamp,title,genres,totalRatingCount
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,79672
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,7058
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,6616
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War,1269
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance,10895


In [8]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   58958.000
mean      424.032
std      2479.745
min         1.000
25%         2.000
50%         6.000
75%        36.750
max     81491.000
Name: totalRatingCount, dtype: float64


In [9]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,timestamp,title,genres,totalRatingCount
0,1,296,5.000,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,79672
1,1,306,3.500,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,7058
2,1,307,5.000,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,6616
3,1,665,5.000,1147878820,Underground (1995),Comedy|Drama|War,1269
4,1,899,3.500,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance,10895


In [10]:
small_rating_popular_movie=rating_popular_movie[:10000]
small_rating_popular_movie.shape

(10000, 7)

In [11]:
movie_features_df=small_rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,66,67,68,69,70,71,72,73,74,75
title,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"'burbs, The (1989)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000
(500) Days of Summer (2009),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...And Justice for All (1979),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10 (1979),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000


In [12]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
movie_features_df.shape

(3223, 75)

In [14]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)

2500


In [15]:
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 20)

In [16]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,66,67,68,69,70,71,72,73,74,75
title,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"'burbs, The (1989)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000
(500) Days of Summer (2009),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...And Justice for All (1979),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10 (1979),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000


In [17]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Scarface (1983):

1: Superman II (1980), with distance of 0.1726436971623504:
2: Underworld (2003), with distance of 0.24622163855559098:
3: Van Helsing (2004), with distance of 0.257919256788797:
4: American Gangster (2007), with distance of 0.3407936598694218:
5: Fast & Furious 6 (Fast and the Furious 6, The) (2013), with distance of 0.3603978509331687:
6: Tourist, The (2010), with distance of 0.3603978509331687:
7: Fantastic Four: Rise of the Silver Surfer (2007), with distance of 0.3603978509331687:
8: Flight (2012), with distance of 0.3603978509331687:
9: Death Race (2008), with distance of 0.3603978509331687:
10: Wedding Planner, The (2001), with distance of 0.3603978509331687:
11: Hotel Transylvania (2012), with distance of 0.3603978509331687:
12: Tinker Tailor Soldier Spy (2011), with distance of 0.3603978509331687:
13: Skyline (2010), with distance of 0.3603978509331687:
14: Daybreakers (2010), with distance of 0.3603978509331687:
15: Grey, The (2012), with